In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.model_selection import train_test_split
import sklearn.preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

#import warnings
#warnings.filterwarnings("ignore")

from scipy import stats
import re

import wrangle as w
import QMCBT_explore_evaluate as ee
#import QMCBT_modeling as m

In [2]:
df = w.get_telco_churn_df()

In [3]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,7033,7034,7035,7036,7037,7038,7039,7040,7041,7042
customer_id,0002-ORFBO,0003-MKNFE,0004-TLHLJ,0011-IGKFF,0013-EXCHZ,0013-MHZWF,0013-SMEOE,0014-BMAQU,0015-UOCOJ,0016-QLJIS,...,9964-WBQDJ,9965-YOKZB,9972-NKTFD,9975-GPKZU,9975-SKRNR,9979-RGMZT,9985-MWVIX,9986-BONCE,9992-UJOEL,9995-HOTOH
payment_type_id,2,2,1,1,2,4,3,4,1,2,...,4,1,3,4,2,2,2,3,2,1
contract_type_id,2,1,1,1,1,1,3,3,1,3,...,3,1,1,3,1,2,1,1,1,3
internet_service_type_id,1,1,2,2,2,1,2,2,1,1,...,3,2,1,3,3,2,2,3,1,1
gender,Female,Male,Male,Male,Female,Female,Female,Male,Female,Female,...,Female,Male,Female,Male,Male,Female,Female,Female,Male,Male
senior_citizen,0,0,0,1,1,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,0
partner,Yes,No,No,Yes,Yes,No,Yes,Yes,No,Yes,...,Yes,No,No,Yes,No,No,No,No,No,Yes
dependents,Yes,No,No,No,No,Yes,No,No,No,Yes,...,No,No,No,Yes,No,No,No,No,No,Yes
tenure,9,9,4,13,3,9,71,63,7,65,...,71,9,28,46,1,7,1,4,2,63
phone_service,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No


In [4]:
df = w.clean_telco(df)

In [5]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,7033,7034,7035,7036,7037,7038,7039,7040,7041,7042
senior_citizen,0,0,0,1,1,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,0
tenure,9,9,4,13,3,9,71,63,7,65,...,71,9,28,46,1,7,1,4,2,63
monthly_charges,65.6,59.9,73.9,98.0,83.9,69.4,109.7,84.65,48.2,90.45,...,24.4,83.85,54.65,19.75,18.9,94.05,70.15,20.95,50.3,59.0
total_charges,593.3,542.4,280.85,1237.85,267.4,571.45,7904.25,5377.8,340.35,5957.9,...,1725.4,790.15,1517.5,856.5,18.9,633.45,70.15,85.5,92.75,3707.6
signup_date,2021-04-21 18:07:34,2021-04-21 18:07:34,2021-09-21 18:07:34,2020-12-21 18:07:34,2021-10-21 18:07:34,2021-04-21 18:07:34,2016-02-21 18:07:34,2016-10-21 18:07:34,2021-06-21 18:07:34,2016-08-21 18:07:34,...,2016-02-21 18:07:34,2021-04-21 18:07:34,2019-09-21 18:07:34,2018-03-21 18:07:34,2021-12-21 18:07:34,2021-06-21 18:07:34,2021-12-21 18:07:34,2021-09-21 18:07:34,2021-11-21 18:07:34,2016-10-21 18:07:34
gender_encoded,0,1,1,1,0,0,0,1,0,0,...,0,1,0,1,1,0,0,0,1,1
partner_encoded,1,0,0,1,1,0,1,1,0,1,...,1,0,0,1,0,0,0,0,0,1
dependents_encoded,1,0,0,0,0,1,0,0,0,1,...,0,0,0,1,0,0,0,0,0,1
phone_service_encoded,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,0
paperless_billing_encoded,1,0,1,1,1,1,1,1,1,1,...,1,1,0,0,0,1,1,0,1,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 25 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   senior_citizen                        7043 non-null   int64         
 1   tenure                                7043 non-null   int64         
 2   monthly_charges                       7043 non-null   float64       
 3   total_charges                         7043 non-null   float64       
 4   signup_date                           7043 non-null   datetime64[ns]
 5   gender_encoded                        7043 non-null   int64         
 6   partner_encoded                       7043 non-null   int64         
 7   dependents_encoded                    7043 non-null   int64         
 8   phone_service_encoded                 7043 non-null   int64         
 9   paperless_billing_encoded             7043 non-null   int64         
 10  